In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import albumentations as A
from pathlib import Path
import os
from tqdm import tqdm
import sklearn.metrics
import PIL.Image as Image, PIL.ImageDraw as ImageDraw, PIL.ImageFont as ImageFont
from matplotlib.font_manager import FontProperties
prop = FontProperties()
plt.style.use('seaborn-dark-palette')
#prop.set_file('../data/kalpurush.ttf')
%matplotlib inline

In [0]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
from IPython.display import SVG
from tensorflow.keras import backend as K
np.random.seed(42)
tf.random.set_seed(42)

TensorFlow 2.x selected.


In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout, Activation, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPool2D, AvgPool2D

In [0]:
dataframe = pd.read_parquet('/content/drive/My Drive/Bengali Grapheme/data/train_labels.parquet')

In [0]:
X = dataframe.drop(['image_id','grapheme_root','vowel_diacritic','consonant_diacritic'],axis=1).values

In [0]:
y = dataframe[['grapheme_root','vowel_diacritic','consonant_diacritic']]

In [0]:
del dataframe

In [0]:
X = X/255

---

## Generators /Train Test Split

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.33, random_state=42)

del X_train, y_train

In [0]:
def model_alexnet():
    
    input_ = Input(shape=(64,64,1))
    
    conv1 = Conv2D(filters=96, kernel_size=(11, 11), strides=4, padding='valid', activation='relu')(input_)
    maxpool1 = MaxPool2D(pool_size=(3, 3), strides=2, padding='valid')(conv1)
    bn1 = BatchNormalization()(maxpool1)
    
    conv2 = Conv2D(filters=256, kernel_size=(5, 5), strides=1, padding='same', activation='relu')(bn1)
    maxpool2 = MaxPool2D(pool_size=(3, 3), strides=2, padding='valid')(conv2)
    bn2 = BatchNormalization()(maxpool2)
    
    conv3 = Conv2D(filters=384, kernel_size=(3, 3), strides=1, padding='same', activation='relu')(bn2)
    drop1 = Dropout(0.5)(conv3)
    conv4 = Conv2D(filters=384, kernel_size=(3, 3), strides=1, padding='same', activation='relu')(drop1)
    drop2 = Dropout(0.5)(conv4)
    conv5 = Conv2D(filters=256, kernel_size=(3, 3), strides=1, padding='same', activation='relu')(drop2)
    
    flat = Flatten()(conv5)
    
    dense_root_1 = Dense(4096,activation='relu')(flat)
    dense_root_2 = Dense(4096,activation='relu')(dense_root_1)
    dense_root_3 = Dense(1000,activation='relu')(dense_root_2)
    output_root = Dense(168,activation='softmax',name='root')(dense_root_3)
    
    dense_vowel_1 = Dense(800,activation='relu')(flat)
    dense_vowel_2 = Dense(600,activation='relu')(dense_vowel_1)
    dense_vowel_3 = Dense(100,activation='relu')(dense_vowel_2)
    output_vowel = Dense(11,activation='softmax',name='vowel')(dense_vowel_3)
    
    dense_consonant_1 = Dense(800,activation='relu')(flat)
    dense_consonant_2 = Dense(600,activation='relu')(dense_consonant_1)
    dense_consonant_3 = Dense(100,activation='relu')(dense_consonant_2)
    output_consonant = Dense(7,activation='softmax',name='consonant')(dense_consonant_3)
    
    model = keras.Model(inputs=[input_],outputs=[output_root,output_vowel,output_consonant])
    
    return model

In [0]:
keras.backend.clear_session()
model_alexnet = model_alexnet()
#keras.utils.plot_model(model_alexnet, '../results/alexnet.png', expand_nested=True, show_shapes=True);

In [0]:
model_alexnet.compile(loss = [keras.losses.sparse_categorical_crossentropy,
                              keras.losses.sparse_categorical_crossentropy,
                              keras.losses.sparse_categorical_crossentropy],
              optimizer = 'Adam',
              metrics=['accuracy'])

In [0]:
history_alexnet = model_alexnet.fit(x_train.reshape(-1,64,64,1),
                                    (y_train['grapheme_root'].values, y_train['vowel_diacritic'].values,
                                     y_train['consonant_diacritic'].values), 
                                    validation_data = (x_valid.reshape(-1,64,64,1), (y_valid['grapheme_root'].values, y_valid['vowel_diacritic'].values,
                                     y_valid['consonant_diacritic'].values)
                                    batch_size=30, epochs=30)

Train on 200840 samples
Epoch 1/30
200840/200840 [==============================] - 78s 389us/sample - loss: 3.4582 - root_loss: 2.4689 - vowel_loss: 0.5642 - consonant_loss: 0.4250 - root_accuracy: 0.3204 - vowel_accuracy: 0.8067 - consonant_accuracy: 0.8558
Epoch 2/30
200840/200840 [==============================] - 79s 392us/sample - loss: 2.8346 - root_loss: 1.9887 - vowel_loss: 0.4875 - consonant_loss: 0.3585 - root_accuracy: 0.4387 - vowel_accuracy: 0.8348 - consonant_accuracy: 0.8789
Epoch 3/30
200840/200840 [==============================] - 78s 388us/sample - loss: 2.4761 - root_loss: 1.7178 - vowel_loss: 0.4372 - consonant_loss: 0.3211 - root_accuracy: 0.5136 - vowel_accuracy: 0.8542 - consonant_accuracy: 0.8919
Epoch 4/30
200840/200840 [==============================] - 78s 388us/sample - loss: 2.2429 - root_loss: 1.5439 - vowel_loss: 0.4031 - consonant_loss: 0.2960 - root_accuracy: 0.5602 - vowel_accuracy: 0.8662 - consonant_accuracy: 0.9011
Epoch 5/30
200840/200840 [======